# Basic Metric

This is the basic comparison of simply comparing how many of the sentences the model got correct in terms of the extracted ground truth

In [2]:
import pandas as pd

In [3]:
ground_truth = pd.read_csv('standup_data.csv')
gemma = pd.read_csv('gemma_answers.csv')

In [28]:
print(ground_truth.columns)

Index(['comedian', 'laugh_start', 'laugh_end', 'sentence'], dtype='object')


In [4]:
ground_truth.head()

,comedian,laugh_start,laugh_end,sentence
0,Donald_Glover,14.268125,19.352170,"I was babysitting this kid once, this mean kid..."
1,Donald_Glover,33.292292,34.182586,The sweetest thing he was allowed was mints. H...
2,Donald_Glover,37.790618,53.441041,So he would steal mints by the handful.
3,Donald_Glover,68.880605,73.824077,And I would take him to the park and I was the...
4,Donald_Glover,85.491608,88.607636,"You know, we'd trade jerk- chicken recipes and..."


In [5]:
gemma.head()

,Comedian,Sentence
0,Anthony_Jeselnik,"When I was a kid, I used to fantasize about ge..."
1,Anthony_Jeselnik,My mom actually should've been on one of the p...
2,Anthony_Jeselnik,"When I was a kid, like nine years old, I'd com..."
3,Anthony_Jeselnik_2,I've never talked to a group of people without...
4,Anthony_Jeselnik_2,"And I know my grandma loved it too, because it..."


In [7]:
import re

def simple_sentence(sentence):
    cleaned_sentence = sentence.lower()
    cleaned_sentence = re.sub(r'[^\w\s]', '', cleaned_sentence)
    return cleaned_sentence

In [77]:
found = {}
correct_guesses = {}

for index, row in ground_truth.iterrows():
    comedian_name = row['comedian']
    sentence1 = row['sentence']   
    simple_truth = simple_sentence(sentence1)
    matching_rows = gemma[gemma['Comedian'] == comedian_name]
    correct_guesses.setdefault(comedian_name, 0)

    if sentence1 not in found:
        found[sentence1] = [comedian_name, False]  

    for index2, row2 in matching_rows.iterrows():
        sentence2 = row2['Sentence']      
        simple_gemma = simple_sentence(sentence2)
        
        if simple_truth == simple_gemma or simple_gemma in simple_truth:
            correct_guesses[comedian_name] += 1
            found[sentence1][0] = comedian_name  
            found[sentence1][1] = True          


In [82]:
for name, score in correct_guesses.items():
    print(name, score)
    print()

Donald_Glover 1

Donald_Glover_2 0

Anthony_Jeselnik 2

Anthony_Jeselnik_2 1

Chelsea_Peretti 0

Chelsea_Peretti_2 2

Louis_CK 4

Louis_CK_2 2

John_Mulaney 1

John_Mulaney_2 1

Ali_Wong 0

Ali_Wong_2 2

Hasan_Minhaj 2

Hasan_Minhaj_2 2

Iliza_Shlesinger 0

Iliza_Shlesinger_2 0

Jim_Gaffigan 1

Jim_Gaffigan_2 0

Joe_List 2

Joe_List_2 0

Jimmy_Yang 0

Jimmy_Yang_2 2

Nate_Bargatze 1

Nate_Bargatze_2 0

Nate_Bargatze_TK 1

Nate_Bargatze_TK_2 1

Russell_Peters 1

Russell_Peters_2 2

Sam_Morril 0

Sam_Morril_2 0

Trevor_Noah 0

Trevor_Noah_2 1

Tom_Segura 3

Tom_Segura_2 0



In [80]:
wrong_guesses = {}

for sentence, (comedian_name, match_found) in found.items():
    if not match_found:
        if comedian_name not in wrong_guesses:
            wrong_guesses[comedian_name] = []
        wrong_guesses[comedian_name].append(sentence)

In [81]:
wrong_guesses

{'Donald_Glover': ["I was babysitting this kid once, this mean kid, and I remember the first time I saw him, I opened the door and there were tears streaming down his face, tears streaming down his face, but he wasn't crying. He wasn't crying.",
  'So he would steal mints by the handful.',
  'And I would take him to the park and I was the only boy there, you know, I was hanging out. You know, they were cool.',
  "You know, we'd trade jerk- chicken recipes and stuff.",
  'And he goes, "Shut up." And she goes, "Don\'t you talk to me like that, I am a grown-up, you will respect me."'],
 'Donald_Glover_2': ['I was hanging out in my neighborhood and this girl was jogging and we started talking about a rape that had happened in the neighborhood, and we were talking about... "Oh, this is awful, this is so messed up. I can\'t believe this happened so close to us."',
  '"Yes, I will. I have no problem."',
  'Uh... Two things, lady.',
  "Number one, he's a rapist.",
  "It's probably pretty high.